In [1]:
import pandas as pd
import os

In [2]:
data_url = "/Users/benjaminzaidel/Desktop/Kaggle/Forex_Pairs"

In [3]:
import os
import pandas as pd

def create_wide_dataframe(data_folder: str) -> pd.DataFrame:
    """
    Reads all .txt files in `data_folder`, each containing
    <DTYYYYMMDD>, <TIME>, <CLOSE>, etc. The resulting DataFrame
    has:
      - A 'time_step' column (int) formed by concatenating date/time (YYYYMMDDHHMMSS).
      - One column per currency pair (derived from filename).
      - Each cell is the <CLOSE> value for that (time_step, currency_pair).
    
    Then sorts rows by the first 8 digits of time_step (date),
    and then by the remaining digits (time).
    """

    dfs_for_merge = []

    for filename in os.listdir(data_folder):
        if filename.endswith(".txt"):
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath)

            required_cols = {"<DTYYYYMMDD>", "<TIME>", "<CLOSE>"}
            if not required_cols.issubset(df.columns):
                print(f"Warning: Missing required columns in '{filename}'. Skipping.")
                continue

            # 1) Create time_step = int(<DTYYYYMMDD><TIME>)
            #    e.g. '20010102' + '230100' -> '20010102230100'
            df["time_step"] = (df["<DTYYYYMMDD>"].astype(str) + 
                               df["<TIME>"].astype(str)).astype(int)
            
            # 2) Derive currency pair from the filename (strip .txt)
            currency_pair = os.path.splitext(filename)[0]  # 'AUDJPY.txt' -> 'AUDJPY'

            # 3) Create a "mini DataFrame" with time_step as index,
            #    and one column named after currency_pair (holding <CLOSE>)
            mini_df = df[["time_step", "<CLOSE>"]].copy()
            mini_df = mini_df.rename(columns={"<CLOSE>": currency_pair})
            mini_df.set_index("time_step", inplace=True)

            # 4) Collect for merging
            dfs_for_merge.append(mini_df)

    if not dfs_for_merge:
        print("No valid data found.")
        return pd.DataFrame(columns=["time_step"])

    # 5) Merge everything side-by-side on time_step
    df_wide = pd.concat(dfs_for_merge, axis=1)

    # 6) time_step is the index. Move it to a column for sorting.
    df_wide.reset_index(inplace=True)

    # 7) Sort time_step as if it's YYYYMMDDHHMMSS:
    #    - first 8 digits -> date_part
    #    - remaining digits -> time_part
    #    - then sort by (date_part, time_part)
    time_str = df_wide["time_step"].astype(str)
    date_part = time_str.str[:8].astype(int)
    time_part = time_str.str[8:].astype(int)

    df_wide["date_part"] = date_part
    df_wide["time_part"] = time_part

    df_wide.sort_values(by=["date_part", "time_part"], inplace=True, ascending=[True, True])

    # 8) Drop the helper columns after sorting
    df_wide.drop(["date_part", "time_part"], axis=1, inplace=True)

    return df_wide

In [5]:
from typing import List, Tuple

def build_rate_matrices(df_wide: pd.DataFrame) -> Tuple[List[np.ndarray], List[int]]:
    """
    Convert each row of the wide df into an adjacency matrix.
    Returns a list of matrices (one per row) and
    the corresponding list of time_steps for reference.
    """
    # 1) Identify all currency pairs
    all_pairs = [col for col in df_wide.columns if col != "time_step"]
    
    # 2) Extract unique currencies
    currency_set = set()
    for pair in all_pairs:
        base = pair[:3]
        quote = pair[3:]
        currency_set.add(base)
        currency_set.add(quote)
    currency_list = sorted(list(currency_set))
    currency_to_idx = {cur: i for i, cur in enumerate(currency_list)}
    
    # 3) We'll build a list of adjacency matrices, one per row
    rate_matrices = []
    time_steps = []

    for _, row in df_wide.iterrows():
        # Initialize adjacency
        n_c = len(currency_list)
        mat = np.zeros((n_c, n_c), dtype=np.float32)
        # set diagonal to 1.0
        np.fill_diagonal(mat, 1.0)
        
        for pair in all_pairs:
            rate = row[pair]
            if pd.isna(rate):
                continue
            base = pair[:3]
            quote = pair[3:]
            
            i = currency_to_idx[base]
            j = currency_to_idx[quote]
            
            mat[i, j] = rate
            
            # if you also want to fill the reciprocal:
            if rate != 0:
                mat[j, i] = 1.0 / rate
        
        rate_matrices.append(mat)
        time_steps.append(row["time_step"])
    
    return rate_matrices, time_steps, currency_list


In [7]:
import gym
import numpy as np
from gym import spaces

class ForexTradingEnv(gym.Env):
    def __init__(self, rate_matrices, currency_list, base_currency='USD'):
        """
        :param rate_matrices: list of shape [n_c, n_c] adjacency matrices
        :param currency_list: list of currency codes, e.g. ['AUD','CAD','CHF','EUR','GBP','JPY','NZD','USD',...]
        :param base_currency: which currency we measure final PnL in
        """
        super(ForexTradingEnv, self).__init__()
        
        self.rate_matrices = rate_matrices
        self.currency_list = currency_list
        self.n_c = len(currency_list)
        
        # Index of the base currency in currency_list
        self.base_idx = currency_list.index(base_currency)
        
        self.num_steps = len(rate_matrices)
        self.current_step = 0
        
        # The agent's holdings in each currency
        self.portfolio = np.zeros(self.n_c, dtype=np.float32)
        self.portfolio[self.base_idx] = 1.0  # start with 1 unit of base currency only
        
        # Action space: choose from among n_c*n_c possible conversions i->j
        self.action_space = spaces.Discrete(self.n_c * self.n_c)
        
        # Observation space: (n_c, n_c) matrix of exchange rates
        # If you want the portfolio included, you can flatten or handle it differently:
        self.observation_space = spaces.Box(
            low=0, high=np.inf,
            shape=(self.n_c, self.n_c),
            dtype=np.float32
        )
    
    def reset(self):
        self.current_step = 0
        self.portfolio[:] = 0
        self.portfolio[self.base_idx] = 1.0  # start with 1 in base
        return self._get_obs()
    
    def _get_obs(self):
        # Possibly just return the adjacency matrix.  If you also want to include
        # the portfolio in the observation, you can flatten them together, e.g.:
        # matrix_flat = self.rate_matrices[self.current_step].flatten()
        # obs = np.concatenate([matrix_flat, self.portfolio], axis=0)
        # But for now, we'll just return the matrix:
        return self.rate_matrices[self.current_step]
    
    def step(self, action):
        i = action // self.n_c
        j = action % self.n_c
        
        done = False
        reward = 0.0
        
        current_matrix = self.rate_matrices[self.current_step]
        old_val_base = self._value_in_base(current_matrix)
        
        # Perform the currency conversion i->j if valid
        if i != j:
            rate = current_matrix[i, j]
            if rate > 0:
                amount_i = self.portfolio[i]
                if amount_i > 0:
                    self.portfolio[i] = 0.0
                    self.portfolio[j] += amount_i * rate
        
        # Move to the next step
        self.current_step += 1
        if self.current_step >= self.num_steps:
            # Episode is done. Final reward is difference between final and initial.
            done = True
            current_matrix = self.rate_matrices[-1]
            final_val_base = self._value_in_base(current_matrix)
            reward = final_val_base - 1.0  # since we started with 1.0
            return None, reward, done, {}
        
        # Otherwise, measure immediate reward as the change in base currency value
        new_matrix = self.rate_matrices[self.current_step]
        new_val_base = self._value_in_base(new_matrix)
        reward = new_val_base - old_val_base
        
        obs = self._get_obs()
        return obs, reward, done, {}
    
    def _value_in_base(self, matrix):
        """
        Convert the entire portfolio to base currency using the given adjacency matrix.
        If rate=0 or NaN, treat it as unconvertible (skipped).
        """
        total_base = 0.0
        for c_idx, amt in enumerate(self.portfolio):
            if c_idx == self.base_idx:
                total_base += amt
            else:
                rate = matrix[c_idx, self.base_idx]
                if rate > 0:
                    total_base += amt * rate
                else:
                    # missing or invalid, treat as 0
                    pass
        return total_base


In [9]:
if __name__ == "__main__":
    # 1) Build your wide DataFrame
    df_wide = create_wide_dataframe(data_url)
    
    # 2) Convert df_wide into adjacency matrices
    rate_matrices, time_steps, currency_list = build_rate_matrices(df_wide)

    # 3) Create environment
    env = ForexTradingEnv(rate_matrices, currency_list, base_currency='USD')
    
    # 4) Quick random-policy run
    obs = env.reset()
    done = False
    total_reward = 0.0
    
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        total_reward += reward
    
    print("Random policy total reward:", total_reward)


KeyboardInterrupt: 